# Create Custom Metric Provider for OpenScale
This notebook 
1. creates a deployment of how the metrics in a custom monitor need to be calculated, a.k.a. custom metric provider 
2. registers the custom metric provider in openscale
3. creates the corresponding monitor definition
4. updates monitor metadata

In [1]:
import os
import wml_sdk_utils as wml_util

In [2]:
confs = {'custom_metrics_generic_edited.py':
            {'function_asset_name':'Generic Metrics Provider Function wendy',
             'function_deployment_name':'Generic Metrics Provider Deployment wendy',
             'openscale_integrated_system_name':"Generic Metrics Provider",
             'openscale_monitor_name':'Generic Metrics',
             'openscale_monitor_id':None,
             'openscale_monitor_defaults':
                {'num_images_total_ground_truth': {'threshold':[5,'lower']},
                 'num_images_total_predicted': {'threshold':[40,'lower']},
                 'num_images_recent_ground_truth': {'threshold':[5,'lower']},
                 'num_images_recent_predicted': {'threshold':[40,'lower']}}
             },
           'custom_metrics_segmentation_edited.py':
            {'function_asset_name':'Segmentation Metrics Provider Function wendy',
             'function_deployment_name':'Segmentation Metrics Provider Deployment wendy',
             'openscale_integrated_system_name':"Segmentation Metrics Provider",
             'openscale_monitor_name':'Segmentation Metrics',
             'openscale_monitor_id':None,
             'openscale_monitor_defaults':
                 {'precision': {'threshold':[60,'lower']},
                 'precision_positive': {'threshold':[0.6,'lower']},
                 'precision_negative': {'threshold':[0.6,'lower']},
                 'recall': {'threshold':[15,'lower']},
                 'recall_positive': {'threshold':[0.15,'lower']},
                 'recall_negative': {'threshold':[0.15,'lower']},
                 'f1': {'threshold':[45,'lower']},
                 'f1_positive': {'threshold':[0.45,'lower']},
                 'f1_negative': {'threshold':[0.45,'lower']},
                 'Dice': {'threshold':[25,'lower']},
                 'Dice_positive': {'threshold':[0.25,'lower']},
                 'Dice_negative': {'threshold':[0.25,'lower']},
                 'IOU': {'threshold':[15,'lower']},
                 'IOU_positive': {'threshold':[0.15,'lower']},
                 'IOU_negative': {'threshold':[0.15,'lower']},
                 'PixAcc': {'threshold':[35,'lower']},
                 'PixAcc_positive': {'threshold':[0.35,'lower']},
                 'PixAcc_negative': {'threshold':[0.35,'lower']}}
            }
          }

In [3]:
path_custom_metrics_script = 'custom_metrics_generic_edited.py'
conf = confs[path_custom_metrics_script]

WML_SPACE_ID = '934341dc-0a71-4d86-9c09-a47261359cca'

WOS_GUID = '00000000-0000-0000-0000-000000000000'

In [4]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)
wml_client.version

'1.0.173'

In [5]:
wml_client.repository.list_functions()

------------------------------------  --------------------------------------------  ------------------------  ------
GUID                                  NAME                                          CREATED                   TYPE
95176c20-64f9-41e7-8804-78d8912864af  Segmentation Metrics Provider Function wendy  2022-05-11T17:32:30.002Z  python
5b21e501-d854-4d15-9b7d-8981554b0c45  generic metrics wendy                         2022-05-03T21:06:26.002Z  python
2286e733-a493-48f3-b02f-e37790a906ac  wml dummy function wendy                      2022-05-03T21:05:43.002Z  python
------------------------------------  --------------------------------------------  ------------------------  ------


## 1. Store and Deploy Custom Metrics Provider in the form of a Deployable Python function

The Python function receives the required variables, such as the datamart_id, monitor_instance_id, monitor_id, monitor_instance_parameters and subscription_id from the Watson OpenScale service when it is invoked by the custom monitor.

In the Python function, add your own logic to compute the custom metrics in the get_metrics method, publish the metrics to the Watson Openscale service and update the status of the run to the finished state in the custom monitor instance.

Update the WOS_CREDENTIALS in the Python function.

In [6]:
function_asset_id = wml_util.function_store(path_custom_metrics_script,wml_client,
                                            function_name=conf['function_asset_name'])
print(function_asset_id)

No function asset found with name Generic Metrics Provider Function wendy.
Function UID = 56679617-03c1-4f4c-9549-e38c1ec979a7
deleting tmp/custom_metrics_generic_edited.py.gz...
56679617-03c1-4f4c-9549-e38c1ec979a7


In [7]:
deployment_id,scoring_url = wml_util.function_deploy(function_asset_id,wml_client,
                                                      function_deployment_name=conf['function_deployment_name'])
print(deployment_id,scoring_url)



#######################################################################################

Synchronous deployment creation for uid: '56679617-03c1-4f4c-9549-e38c1ec979a7' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0dac9ec0-0fab-4e79-8d8e-65a6ddd39022'
------------------------------------------------------------------------------------------------


0dac9ec0-0fab-4e79-8d8e-65a6ddd39022 https://internal-nginx-svc:12443/ml/v4/deployments/0dac9ec0-0fab-4e79-8d8e-65a6ddd39022/predictions?version=2022-05-11


## 2. Register in OpenScale

In [8]:
from ibm_watson_openscale import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
import wos_sdk_utils as wos_util

from datetime import datetime, timezone, timedelta
import uuid

In [9]:
wos_client = wos_util.get_client()
wos_client.version

'3.0.17'

Update the custom metrics deployment URL, which is created during the Python function creation in the integrated system. Watson OpenScale invokes the deployment URL at runtime to compute the custom metrics.

You must define the authentication type based on the communication with custom metrics deployment. Watson OpenScale supports 2 types of authentication: basic and bearer. If custom metrics deployment accepts the basic authentication type, then provide auth_type=basic otherwise use auth_type=bearer.

In [10]:
# Delete existing custom metrics provider integrated systems if present
wos_util.integrated_system_delete(conf['openscale_integrated_system_name'],wos_client)

Deleted integrated system Generic Metrics Provider


In [11]:
credentials = {}
with open(path_custom_metrics_script,'r') as f:
    for line in f:
        if "os.environ['USERNAME'] = " in line:
            credentials['username'] = eval(line.replace("os.environ['USERNAME'] = ",'').strip())
        elif "os.environ['APIKEY'] = " in line:
            credentials['api_key'] = eval(line.replace("os.environ['APIKEY'] = ",'').strip())
        else:
            pass

assert 'username' in credentials and 'api_key' in credentials, 'Either parsing has issue or the information is not included in the script'

In [12]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=conf['openscale_integrated_system_name'],
    description=conf['openscale_integrated_system_name'],
    type="custom_metrics_provider",
    credentials= {"auth_type":"bearer",
                  "token_info": {
                      "url": "{}/icp4d-api/v1/authorize".format(os.environ['RUNTIME_ENV_APSX_URL']),
                      "headers": {"Content-Type": "application/json",
                                  "Accept": "application/json"},
                      "payload": {'username':credentials['username'],
                                   'api_key':credentials['api_key']},
                      "method": "post"}
                 },
    connection={"display_name": conf['openscale_integrated_system_name'],
                "endpoint": scoring_url
    }).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(integrated_system_id)

33ccf3df-3812-47ee-a4c4-d78c0fb8bbde


## 3. Setup custom monitor definition

In [13]:
monitor_id = wos_util.monitor_definition_create(conf['openscale_monitor_name'],conf['openscale_monitor_defaults'],wos_client,overwrite=True)
print(monitor_id)

Deleted existing monitor definition Generic Metrics (generic_metrics)



 Waiting for end of adding monitor definition generic_metrics 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------


generic_metrics


In [14]:
wos_client.monitor_definitions.show()

generic_metrics,Generic Metrics,"['num_images_total_ground_truth', 'num_images_total_predicted', 'num_images_recent_ground_truth', 'num_images_recent_predicted']"
segmentation_metrics,Segmentation Metrics,"['precision', 'precision_positive', 'precision_negative', 'recall', 'recall_positive', 'recall_negative', 'f1', 'f1_positive', 'f1_negative', 'Dice', 'Dice_positive', 'Dice_negative', 'IOU', 'IOU_positive', 'IOU_negative', 'PixAcc', 'PixAcc_positive', 'PixAcc_negative']"
dummy_monitor_example_wendy,dummy monitor example wendy,"['sensitivity', 'specificity']"
generic_monitor_example_wendy,generic monitor example wendy,"['num_images_recent_ground_truth', 'num_images_recent_predicted', 'num_images_total_ground_truth', 'num_images_total_predicted']"
dummy_monitor_wendy,dummy monitor wendy,"['specificity', 'sensitivity']"
segmentation_metrics_rich_2,Segmentation Metrics Rich 2,"['precision', 'precision_positive', 'precision_negative', 'recall', 'recall_positive', 'recall_negative', 'f1', 'f1_positive', 'f1_negative', 'Dice', 'Dice_positive', 'Dice_negative', 'IOU', 'IOU_positive', 'IOU_negative', 'PixAcc', 'PixAcc_positive', 'PixAcc_negative', 'Num_Images']"
segmentation_metrics_rich,Segmentation Metrics Rich,"['precision', 'precision_positive', 'precision_negative', 'recall', 'recall_positive', 'recall_negative', 'f1', 'f1_positive', 'f1_negative', 'Dice', 'Dice_positive', 'Dice_negative', 'IOU', 'IOU_positive', 'IOU_negative', 'PixAcc', 'PixAcc_positive', 'PixAcc_negative', 'Num_Images']"
segmentation_accuracy,Segmentation Accuracy,['dice_score']
segmentation_accuracy_metrics,Segmentation Accuracy Metrics,['Dice Score']
assurance,Assurance,"['Uncertainty', 'Confidence']"


Note: First 10 records were displayed.


## Update Metadata File

In [15]:
metadata = {monitor_id:
            {'integrated_system_id':integrated_system_id,
             'wml_deployment_id':deployment_id}}

metadata

{'generic_metrics': {'integrated_system_id': '33ccf3df-3812-47ee-a4c4-d78c0fb8bbde',
  'wml_deployment_id': '0dac9ec0-0fab-4e79-8d8e-65a6ddd39022'}}

In [16]:
# wml_util.metadata_yml_add(metadata,wml_client,metadata_type='monitor')
wml_util.metadata_yml_add(metadata,wml_client,metadata_type='monitor',overwrite=True)

Successfully saved data asset content to file: 'tmp/monitor_metadata.yml'
Writing new metadata in..
Creating data asset...
SUCCESS
Finished publishing tmp/monitor_metadata.yml as monitor_metadata.yml
Finished deleting existing old assets with the same name
deleting tmp/monitor_metadata.yml...


In [17]:
wml_util.metadata_yml_load(wml_client,metadata_type='monitor')

Successfully saved data asset content to file: 'tmp/monitor_metadata.yml'
deleting tmp/monitor_metadata.yml...


{'MONITOR_ID': {'integrated_system_id': '', 'wml_deployment_id': ''},
 'generic_metrics': {'integrated_system_id': '33ccf3df-3812-47ee-a4c4-d78c0fb8bbde',
  'wml_deployment_id': '0dac9ec0-0fab-4e79-8d8e-65a6ddd39022'},
 'segmentation_metrics': {'integrated_system_id': '29a719d8-0840-4c2d-8414-c93fa9035b41',
  'wml_deployment_id': 'f7b120bd-a9bf-48fa-9cef-404e16531b70'}}